# Install Modules

# Imports

In [116]:
# 1. Basic Python Imports
import pandas as pd
import numpy as np
import pickle as pi
import os

# 2. API PULL
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

# 3. API Keys
from dotenv import load_dotenv


In [117]:
# VARIABLES

# Load the environment variables from the .env file
coin_market_api_key = '3486d5cf-6e5b-443f-8f9a-140421f63bb3'


In [118]:


coin_list_number =list(range(1,100))
coin_string =','.join([str(i) for i in coin_list_number])
replace_ids = ['11,','21,','84,']

for bad_coin_id in replace_ids:
    coin_string = coin_string.replace(bad_coin_id,'')


In [119]:
url = f'https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest'



parameters = {
    'id':coin_string,
    #'aux':"num_market_pairs,cmc_rank,date_added,tags,max_supply,circulating_supply,total_supply,is_active,is_fiat"
}


headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY':coin_market_api_key,
} 

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  #print(json.dumps(data, indent=4, sort_keys=True))
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

data
coin_data = pd.DataFrame.from_dict(data["data"])
crypto_df = coin_data.T


#only take relevant data
crypto_df



,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,is_active,platform,cmc_rank,is_fiat,self_reported_circulating_supply,self_reported_market_cap,last_updated,quote
1,1,Bitcoin,BTC,bitcoin,9398,2013-04-28T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",21000000,19030943,19030943,1,None,1,0,None,None,2022-05-05T02:14:00.000Z,"{'USD': {'price': 39681.88598847886, 'volume_2..."
2,2,Litecoin,LTC,litecoin,766,2013-04-28T00:00:00.000Z,"[mineable, pow, scrypt, medium-of-exchange, bi...",84000000,70214818.777915,84000000,1,None,22,0,None,None,2022-05-05T02:14:00.000Z,"{'USD': {'price': 106.23843820275442, 'volume_..."
3,3,Namecoin,NMC,namecoin,7,2013-04-28T00:00:00.000Z,"[mineable, pow, sha-256, platform]",None,14736400,14736400,1,None,636,0,None,None,2022-05-05T02:14:00.000Z,"{'USD': {'price': 2.082336686263321, 'volume_2..."
4,4,Terracoin,TRC,terracoin,2,2013-04-28T00:00:00.000Z,"[mineable, pow, sha-256, masternodes]",42000000,22935396.430361,22935396.430361,1,None,1907,0,None,None,2022-05-05T02:14:00.000Z,"{'USD': {'price': 0.02230121992552512, 'volume..."
5,5,Peercoin,PPC,peercoin,11,2013-04-28T00:00:00.000Z,"[mineable, hybrid-pow-pos, sha-256, medium-of-...",None,27523832.697212,27523832.697212,1,None,859,0,None,None,2022-05-05T02:14:00.000Z,"{'USD': {'price': 0.46392311206044423, 'volume..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,YbCoin,YBC,ybcoin,0,2014-01-16T00:00:00.000Z,[],None,3020627.191411,3020627.191411,0,None,None,0,None,None,2017-09-29T11:59:02.000Z,"{'USD': {'price': 0.96595931, 'volume_24h': 0,..."
96,96,Extremecoin,EXC,extremecoin,0,2014-01-16T00:00:00.000Z,[mineable],None,3679580.8046,3679580.8046,0,None,None,0,None,None,2016-03-25T17:59:04.000Z,"{'USD': {'price': 0.01554259, 'volume_24h': 0,..."
97,97,eToken,ETOK,etoken,0,2014-01-16T00:00:00.000Z,[mineable],None,304607,304607,0,None,None,0,None,None,2014-08-03T03:49:04.000Z,"{'USD': {'price': 0.00058956, 'volume_24h': 0,..."
98,98,Grain,GRA,grain,0,2014-01-20T00:00:00.000Z,[mineable],None,970635753,970635753,0,None,None,0,None,None,2015-01-09T16:34:06.000Z,"{'USD': {'price': 1e-07, 'volume_24h': 0, 'vol..."


In [120]:
# dive into 'quote'
quotes_dict = crypto_df[['name', 'quote']].copy()
# drop quote for later concat
crypto_df = crypto_df.drop(columns = ['quote'])

In [121]:
# pull 'price' column from quotes dictionary
usd = quotes_dict["quote"].apply(pd.Series)
price = usd['USD'].apply(pd.Series)
#quotes_df = pd.concat([usd, price], join = 'inner')

In [124]:
quotes_df = pd.concat([crypto_df, price], axis = 1, join = 'inner')
display(quotes_df.columns)
display(quotes_df.head())

Index(['id', 'name', 'symbol', 'slug', 'num_market_pairs', 'date_added',
       'tags', 'max_supply', 'circulating_supply', 'total_supply', 'is_active',
       'platform', 'cmc_rank', 'is_fiat', 'self_reported_circulating_supply',
       'self_reported_market_cap', 'last_updated', 'price', 'volume_24h',
       'volume_change_24h', 'percent_change_1h', 'percent_change_24h',
       'percent_change_7d', 'percent_change_30d', 'percent_change_60d',
       'percent_change_90d', 'market_cap', 'market_cap_dominance',
       'fully_diluted_market_cap', 'last_updated'],
      dtype='object')

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,...,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,last_updated
1,1,Bitcoin,BTC,bitcoin,9398,2013-04-28T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",21000000,19030943,19030943,...,0.094454,4.593825,1.030522,-14.948122,0.512365,6.652097,7.551837e+11,41.8539,8.333196e+11,2022-05-05T02:14:00.000Z
2,2,Litecoin,LTC,litecoin,766,2013-04-28T00:00:00.000Z,"[mineable, pow, scrypt, medium-of-exchange, bi...",84000000,70214818.777915,84000000,...,-0.173872,5.954326,6.009645,-15.305277,0.996048,-2.811025,7.459513e+09,0.4134,8.924029e+09,2022-05-05T02:14:00.000Z
3,3,Namecoin,NMC,namecoin,7,2013-04-28T00:00:00.000Z,"[mineable, pow, sha-256, platform]",None,14736400,14736400,...,0.095139,3.398095,-1.072869,-16.756447,-5.920031,24.107577,3.068615e+07,0.0000,3.068615e+07,2022-05-05T02:14:00.000Z
4,4,Terracoin,TRC,terracoin,2,2013-04-28T00:00:00.000Z,"[mineable, pow, sha-256, masternodes]",42000000,22935396.430361,22935396.430361,...,0.094454,3.488961,-0.909202,-2.043662,36.458635,-33.695420,5.114873e+05,0.0000,9.366512e+05,2022-05-05T02:14:00.000Z
5,5,Peercoin,PPC,peercoin,11,2013-04-28T00:00:00.000Z,"[mineable, hybrid-pow-pos, sha-256, medium-of-...",None,27523832.697212,27523832.697212,...,-0.662375,3.073591,0.033262,-29.055044,-14.287343,-20.117025,1.276894e+07,0.0007,1.276894e+07,2022-05-05T02:14:00.000Z
